In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('movie_list.csv')

data.columns = ['Title','Director','Screenwriter','Synopsis','Score','NumOfScore','Actors','ScoresOfComment','Comment','Genre','Country','Viewer','RunningTime','Adult']
data = data.dropna(axis=0).reset_index(drop=True)

data.head()

# Motivation

사용자 요구 & ***추천***

- SQL 기반의 DB와 쿼리로 추천
- 근데 filtering하다보니.... -> 이거 excel filtering이랑 다를게 뭐지?
  => LLM을 유의미하게 써보자!

근데 계속 VectorDB에 생각이 좀 갇혀버린 느낌도....\
[평점, 관람객 수, 리뷰 수] 인기도(goodness) <-> 적합(fitneess) [**리뷰**, 줄거리]

vector search가 안되니까....

### 그물을 펼치는 것처럼
요구 ------------------> 영화\
요구 -------------------- 영화\
--> 유저태그1 == 영화태그1 <--\
--> 유저태그2 == 영화태그2 <--\
--> 유저태그3 == 영화태그3 <--

# 리뷰

영화 리뷰를 5개 이하의 해시태그로 요약

In [21]:
import openai

from langchain_openai import ChatOpenAI
from langchain import LLMChain, PromptTemplate


## Load API
# load_dotenv()
# api_key = os.getenv('OPENAI_API_KEY')

hashtagger = ChatOpenAI(model = 'gpt-4o-mini', temperature = 0)

In [40]:
prompt_tagging = """영화리뷰를 해시태그로 요약해줘. 해시태그 개수는 5개 이하로 필요한 만큼 써. 해시태그는 띄어쓰기 없이 붙여서 반환해줘

리뷰: 그 터널을 무사히 빠져나온 모두에게
해시태그: #극복#희망

리뷰: 사람은 자신의 크기에 맞는 사랑을 한다
해시태그: #사랑#자아수용

리뷰: 샘이라는 여자가 내 인생에 있다면, 나는 내 인생을 통째로 그녀에게 헌납하고 싶다. 패트릭이라는 친구가 내 인생에 있다면, 그와 같은 열정으로 살고 싶다.
해시태그: #헌신#열정#우정

리뷰: '그거 아세요? 이 영화에는 배경음악이 단 일초도 쓰이지 않았다는것||피튀기는 젊은 세상. 그러한 황량하고 잔인한 사막 한가운데서 자신이 할 수 있는 것이 아직 존재한다고 믿는 보안관. 적막 속에서 그렇지 않음을 깨달았을 때, 웬 힘없는 노인이 한 명 서있다.||불행은 단발머리를 하고 한 손엔 동전을 쥔 채 예고없이 나타난다.||타고난 재기, 뛰어난 테크닉, 그리고 어둡고 깊은 우물 하나.||내 수준이 아직 이정도까진 아닌듯 하다. 걸작이라는 영화를 보고 난 왜 평범 그 이상의것을 느끼지 못하는걸까ㅜㅜ||"세월은 막을 수 없는 거야. 너를 기다려주지 않을 거고, 그게 바로 \'허무\'야."||온통 돈에 대한 욕심들로 가득차 있다. 옷을 벗어준 두 어린 아이들 조차도 돈 때문에 다툰다.||살인마가 쓸데없이 머리는 귀엽다;||적막이 흐르면 더 무서운 영화||우리에게도 언젠가는 쉬거가 찾아오겠죠. 쉬거는 곧 "죽음" 이니까.'
해시태그: #허무#적막#재기#돈#죽음

리뷰: {review}
해시태그:"""

hastag_prompt = PromptTemplate(
    template  = prompt_tagging,
    input_variables = ['review']
)

tagging = LLMChain(
    llm = hashtagger,
    prompt = hastag_prompt
)

result = tagging.invoke({"review":data.loc[3, 'Comment']})
result['text']

'#허무#적막#재기#돈#죽음'

In [42]:
for i in data.index:

    result = tagging.invoke({"review":data.loc[i, 'Comment']})
    data.loc[i,'ReviewTag'] = result['text']

# 종합 컬럼

여기에 sql like 쿼리 먹일겁니다.\
감독, 각본가, 장르, 배우, 리뷰태그

single turn이니까 가능한거긴 한데\
1) 유저의 request를 감독, 각본가, 장르, 배우, 감성 관련해서 5개의 해시태그로 변형
2) 5번 sql like 쿼리

In [48]:
infos = ['Title','Director','Screenwriter','Actors','Genre','ReviewTag']
data['Overall'] = data[infos].sum(axis=1)

In [54]:
data.head()

,Title,Director,Screenwriter,Synopsis,Score,NumOfScore,Actors,ScoresOfComment,Comment,Genre,Country,Viewer,RunningTime,Adult,ReviewTag,Overall
0,월플라워,스티븐 크보스키,스티븐 크보스키,말 못할 트라우마를 가지고 자신만의 세계에 갇혀있던 ‘찰리’는 고등학교 신입생이 돼...,3.8,305000.0,에즈라 밀러||로건 메먼||엠마 왓슨,4.0||5.0||4.0||2.0||4.0||4.0||3.5||4.5||3.5||4.5,그 터널을 무사히 빠져나온 모두에게||사람은 자신의 크기에 맞는 사랑을 한다||샘이...,드라마/로맨스,미국,1.7,102.0,0.0,#청춘#사랑#극복#우정#허무,월플라워스티븐 크보스키스티븐 크보스키에즈라 밀러||로건 메먼||엠마 왓슨드라마/로맨...
1,기생충,봉준호,봉준호||한진원,“폐 끼치고 싶진 않았어요” 전원백수로 살 길 막막하지만 사이는 좋은 기택(송강호)...,4.3,1265000.0,박소담||송강호||조여정||최우식||이선균||장혜진||이정은,5.0||4.5||5.0||0.5||4.5||5.0||4.0||5.0||5.0||5.0,황금종려상 받은 영화를 자막없이 볼 수 있는 행복 190529||상승과 하강으로 명...,드라마,한국,1031.0,131.0,0.0,#계급#무기력#생존본능#아이러니#기생충,기생충봉준호봉준호||한진원박소담||송강호||조여정||최우식||이선균||장혜진||이정...
2,메멘토,크리스토퍼 놀란,크리스토퍼 놀란,"아내가 살해당한 후, 10분밖에 기억 못하는 단기기억상실증에 걸린 남자가 사진, 메...",4.1,641000.0,조 판톨리아노||가이 피어스||캐리 앤 모스,5.0||5.0||4.5||5.0||5.0||5.0||5.0||4.0||5.0||4.0,사람은 보고싶은 것만 보고 듣고싶은 것만 듣고 믿고 싶은 것만 믿고 기억하고 싶은 ...,미스터리/스릴러,미국,19.0,113.0,0.0,#기억#역설#편집#해석#창의성,메멘토크리스토퍼 놀란크리스토퍼 놀란조 판톨리아노||가이 피어스||캐리 앤 모스미스터...
3,노인을 위한 나라는 없다,에단 코엔||조엘 코엔,에단 코엔||조엘 코엔,르웰린 모스(조쉬 브롤린)는 총격전이 벌어진 끔찍한 현장에서 우연히 이백만 달러가 ...,4.0,349000.0,토미 리 존스||하비에르 바르뎀||조쉬 브롤린,5.0||5.0||5.0||5.0||3.5||5.0||5.0||4.5||5.0||5.0,그거 아세요? 이 영화에는 배경음악이 단 일초도 쓰이지 않았다는것||피튀기는 젊은 ...,범죄/드라마/스릴러,미국,9.0,122.0,1.0,#허무#적막#재기#돈#죽음,노인을 위한 나라는 없다에단 코엔||조엘 코엔에단 코엔||조엘 코엔토미 리 존스||...
4,박화영,이환,이환,“니들은 나 없으면 어쩔 뻔 봤냐?” 이름: 박화영 나이: 18 직업: 고등학생 가...,3.3,142000.0,김가희||강민아||이재균,3.0||3.0||3.0||3.0||4.5||2.5||3.5||3.0||3.0||2.5,자극적인 장면들을 자꾸 나열한다고 해서 메시지가 더 잘 전달되는 건 아닌데..||불...,드라마,한국,0.6,112.0,1.0,#현실#폭력#고어#사랑#비참,박화영이환이환김가희||강민아||이재균드라마#현실#폭력#고어#사랑#비참


In [50]:
data.to_csv('movie_list.csv', index=False)

# 유저 요구사항 해시태그로 만들기

유저의 요구사항을 gpt-api로 만들기


In [53]:
user_tagger = ChatOpenAI(model= 'gpt-4o-mini', temperature= 0)

usertag_prompt = """영화 추천할거야. 유저의 요청을 감독, 각본가, 장르, 배우, 감성 관련해서 3개이하의 키워드로 변형해줘. 각 키워드는 ||로 구분해줘

user: 레이첼 맥아담스 나오는 영화 추천해줘
answer: 레이첼 맥아담스

user: 신나는 블록버스터 영화 추천해줘
answer: 블록버스터

user: '노트북'이랑 비슷한 영화 추천해줘
answer: 로맨스||레이첼 맥아담스||라이언 고슬링

user: 스타워즈 나 너무 좋아하는데. 비슷한 영화 추천해줘.
answer: SF||판타지

user: 탑건이랑 비슷한 영화 추천해줘.
answer: 톰 크루즈||액션||전투기

user: {query}
answer: """


usertag_template = PromptTemplate(
    template  = usertag_prompt,
    input_variables = ['query']
)

query_making = LLMChain(
    llm = user_tagger,
    prompt = usertag_template
)

result = query_making.invoke(input())
print(result['text'])

액션


In [52]:
result

{'query': '액션영화 추천해줘', 'text': 'keyword: 액션'}